In [183]:
import numpy as np
import huffman
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
import tensorflow_datasets as tfds
import glob
import os
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.datasets import fetch_20newsgroups

In [6]:
texto = "este es un ejemplo de codificación de Huffman"
frequencies = Counter(texto)

In [10]:
codigo_huffman = huffman.codebook(frequencies.items())
texto_codificado = " || ".join(codigo_huffman[char] for char in texto)

print("Texto codificado:", texto_codificado)

# Paso 4: Decodificar el texto codificado
# Crear un diccionario inverso para decodificar
codigo_inverso = {v: k for k, v in codigo_huffman.items()}
texto_decodificado = ""
temp = ""
for bit in texto_codificado:
    temp += bit
    if temp in codigo_inverso:
        texto_decodificado += codigo_inverso[temp]
        temp = ""

print("Texto decodificado:", texto_decodificado)

Texto codificado: 110 || 0100 || 111101 || 110 || 101 || 110 || 0100 || 101 || 0010 || 0111 || 101 || 110 || 111100 || 110 || 0011 || 00010 || 01010 || 0000 || 101 || 1110 || 110 || 101 || 0110 || 0000 || 1110 || 1001 || 1000 || 1001 || 0110 || 11111 || 0110 || 1001 || 00011 || 0111 || 101 || 1110 || 110 || 101 || 01011 || 0010 || 1000 || 1000 || 0011 || 11111 || 0111
Texto decodificado: e


In [13]:
def preprocess_text(text, seq_length=40):
    chars = sorted(set(text))
    char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
    idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}
    vocab_size = len(chars)

    # Crear secuencias de entrada y salida
    input_seq = []
    target_seq = []

    for i in range(len(text) - seq_length):
        in_seq = text[i : i + seq_length]
        out_seq = text[i + seq_length]
        input_seq.append([char_to_idx[char] for char in in_seq])
        target_seq.append(char_to_idx[out_seq])

    # Convertir listas de índices en matrices de entrada y salida adecuadas para el entrenamiento
    input_seq = tf.keras.preprocessing.sequence.pad_sequences(
        input_seq, maxlen=seq_length, padding="pre"
    )
    target_seq = tf.keras.utils.to_categorical(target_seq, num_classes=vocab_size)

    return input_seq, target_seq, vocab_size, char_to_idx, idx_to_char

In [27]:
# Datos de ejemplo
text = """ Hola, ¿cómo estás? Soy un ejemplo de texto que se utilizará para entrenar un modelo de lenguaje.
"""  
seq_length = 10  
input_seq, target_seq, vocab_size, char_to_idx, idx_to_char = preprocess_text(
    text, seq_length
)
print("Input sequence shape:", input_seq)
print("Target sequence shape:", target_seq)
print("Vocabulary size:", idx_to_char)
# Construir el modelo
model = Sequential(
    [
        Embedding(input_dim=vocab_size, output_dim=64, input_length=seq_length),
        LSTM(128),
        Dense(vocab_size, activation="softmax"),
    ]
)

# Compilar el modelo
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Entrenar el modelo
model.fit(input_seq, target_seq, epochs=100, verbose=2)

Input sequence shape: [[ 1  0  4 ... 15  8 16]
 [ 0  4  1 ...  8 16  1]
 [ 4  1  7 ... 16  1 22]
 ...
 [ 1 14  9 ... 18 18  5]
 [14  9  1 ... 18  5  6]
 [ 9  1  5 ...  5  6  5]]
Target sequence shape: [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
Vocabulary size: {0: '\n', 1: ' ', 2: 'A', 3: 'P', 4: 'Y', 5: 'a', 6: 'b', 7: 'c', 8: 'd', 9: 'e', 10: 'g', 11: 'h', 12: 'i', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'z', 24: 'á', 25: 'é', 26: 'í'}
Epoch 1/100


/Users/marcofura/anaconda3/envs/fundamentos_matematicos/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


7/7 - 1s - 132ms/step - accuracy: 0.1618 - loss: 3.2793
Epoch 2/100
7/7 - 0s - 10ms/step - accuracy: 0.1863 - loss: 3.1925
Epoch 3/100
7/7 - 0s - 11ms/step - accuracy: 0.1863 - loss: 2.9225
Epoch 4/100
7/7 - 0s - 10ms/step - accuracy: 0.1912 - loss: 2.8323
Epoch 5/100
7/7 - 0s - 11ms/step - accuracy: 0.1422 - loss: 2.7818
Epoch 6/100
7/7 - 0s - 9ms/step - accuracy: 0.2157 - loss: 2.7413
Epoch 7/100
7/7 - 0s - 9ms/step - accuracy: 0.1863 - loss: 2.7347
Epoch 8/100
7/7 - 0s - 9ms/step - accuracy: 0.1863 - loss: 2.7149
Epoch 9/100
7/7 - 0s - 11ms/step - accuracy: 0.1912 - loss: 2.6929
Epoch 10/100
7/7 - 0s - 9ms/step - accuracy: 0.1961 - loss: 2.6745
Epoch 11/100
7/7 - 0s - 13ms/step - accuracy: 0.1961 - loss: 2.6526
Epoch 12/100
7/7 - 0s - 9ms/step - accuracy: 0.1961 - loss: 2.6338
Epoch 13/100
7/7 - 0s - 9ms/step - accuracy: 0.2451 - loss: 2.5988
Epoch 14/100
7/7 - 0s - 11ms/step - accuracy: 0.2500 - loss: 2.5682
Epoch 15/100
7/7 - 0s - 11ms/step - accuracy: 0.2941 - loss: 2.5319
Epoch 

In [28]:
def predict_next_char(model, input_text, char_to_idx, idx_to_char):
    input_indices = [
        char_to_idx[ch] for ch in input_text[-seq_length:]
    ]  # Asegurarse de usar solo los últimos 'seq_length' caracteres
    input_indices = tf.keras.preprocessing.sequence.pad_sequences(
        [input_indices], maxlen=seq_length, padding="pre"
    )
    predictions = model.predict(input_indices)[0]
    return {idx_to_char[i]: prob for i, prob in enumerate(predictions)}


# Ejemplo de uso
context = "[]Hola"
predicted_probs = predict_next_char(model, context, char_to_idx, idx_to_char)
print(predicted_probs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
{'\n': 2.4446226e-06, ' ': 0.00012292077, 'A': 3.9709444e-06, 'P': 0.00018478041, 'Y': 0.00021594099, 'a': 0.002025231, 'b': 0.0045404113, 'c': 2.0484175e-07, 'd': 4.2705764e-05, 'e': 0.9816928, 'g': 2.1880491e-07, 'h': 2.7613967e-05, 'i': 0.000121175595, 'l': 0.0043668956, 'm': 0.0023769338, 'n': 2.608391e-05, 'o': 5.560564e-06, 'q': 2.5820671e-05, 'r': 0.0002562776, 's': 0.0029980824, 't': 2.9368462e-05, 'u': 0.00015539383, 'v': 0.00076011376, 'z': 7.896544e-07, 'á': 5.8225937e-06, 'é': 1.0626007e-05, 'í': 1.7690279e-06}


In [171]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),categories=['alt.atheism'])
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),categories=['alt.atheism'])

In [174]:
len(newsgroups_train.data), len(newsgroups_test.data)

(480, 319)

In [156]:
def load_texts(directory):
    texts = []
    # Recorre todos los archivos en el directorio
    for filename in glob.glob(os.path.join(directory, "*")):
        if os.path.isfile(filename):  # Check if the path is a file
            with open(filename, "r", encoding="utf-8", errors="ignore") as file:
                texts.append(file.read())
    return texts


# Suponiendo que '20news-bydate-train' es tu directorio descomprimido
train_dir = "./20news-bydate/20news-bydate-train/alt.atheism"
test_dir = "./20news-bydate/20news-bydate-test/alt.atheism"
texts = load_texts(train_dir)
texts_test = load_texts(test_dir)

In [157]:
len(texts)

480

In [160]:
def create_sequences(texts, seq_length=40):
    input_chars = []
    output_char = []
    for text in texts:
        for i in range(len(text) - seq_length):
            in_seq = text[i:i + seq_length]
            out_seq = text[i + seq_length]
            input_chars.append(in_seq)
            output_char.append(out_seq)
    return input_chars, output_char

In [216]:
seq_length = 20
texts_train_part = newsgroups_train.data[:100]
texts_test_part = newsgroups_test.data[:25]
input_seqs, output_seqs = create_sequences(texts_train_part, seq_length)
input_seqs_test, output_seqs_test = create_sequences(texts_test_part, seq_length)

In [177]:
len(input_seqs)

89538

In [165]:
len(input_seqs)

172544

In [217]:

# Tokenización
tokenizer = Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts(newsgroups_train.data)  # Ajuste del tokenizer al corpus completo
vocab_size = len(tokenizer.word_index) + 1

# Convertir textos a secuencias numéricas
input_seqs = tokenizer.texts_to_sequences(input_seqs)
input_seqs = pad_sequences(input_seqs, maxlen=seq_length, truncating='pre')

# Preparar datos de salida (etiquetas) usando one-hot encoding
output_seqs = tokenizer.texts_to_sequences(output_seqs)
output_seqs = to_categorical(output_seqs, num_classes=vocab_size)

# Test
input_seqs_test = tokenizer.texts_to_sequences(input_seqs_test)
input_seqs_test = pad_sequences(input_seqs_test, maxlen=seq_length, truncating='pre')

# Preparar datos de salida (etiquetas) usando one-hot encoding
output_seqs_test = tokenizer.texts_to_sequences(output_seqs_test)
output_seqs_test = to_categorical(output_seqs_test, num_classes=vocab_size)

# Construcción del modelo LSTM
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
    LSTM(512,return_sequences=True),
    Dropout(0.5),
    LSTM(256),
    Dense(256, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

# Compilación del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


# Entrenamiento del modelo
model.fit(input_seqs, output_seqs, epochs=50, batch_size=64,validation_data=(input_seqs_test, output_seqs_test),callbacks=[early_stopping,model_checkpoint])

/Users/marcofura/anaconda3/envs/fundamentos_matematicos/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_48 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_110 (LSTM)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_65 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_111 (LSTM)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 148s 105ms/step - accuracy: 0.2047 - loss: 3.0392 - val_accuracy: 0.4048 - val_loss: 2.2121
Epoch 2/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 148s 105ms/step - accuracy: 0.3816 - loss: 2.1994 - val_accuracy: 0.4769 - val_loss: 1.9456
Epoch 3/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 145s 104ms/step - accuracy: 0.4563 - loss: 1.9095 - val_accuracy: 0.5153 - val_loss: 1.8083
Epoch 4/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 143s 102ms/step - accuracy: 0.5050 - loss: 1.7053 - val_accuracy: 0.5302 - val_loss: 1.7447
Epoch 5/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 146s 104ms/step - accuracy: 0.5349 - loss: 1.5898 - val_accuracy: 0.5446 - val_loss: 1.7153
Epoch 6/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 152s 109ms/step - accuracy: 0.5595 - loss: 1.4895 - val_accuracy: 0.5536 - val_loss: 1.7267
Epoch 7/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 160s 114ms/step - accuracy: 0.5811 - loss: 1.4052 - val_accuracy: 0.5588 - val_loss: 1.7368
Epoch 8/50
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 169s 120ms/step - ac

In [159]:
vocab_size

122

In [88]:
def predict_next_char(model, tokenizer, input_text, seq_length):
    # Paso 1: Convertir la cadena de entrada en una secuencia de enteros
    input_seq = tokenizer.texts_to_sequences([input_text])[0]
    
    # Paso 2: Ajustar la longitud de la secuencia
    input_seq = pad_sequences([input_seq], maxlen=seq_length, truncating='pre')
    
    # Paso 3: Hacer la predicción
    prediction = model.predict(input_seq, verbose=0)[0]
    
    # Paso 4: Convertir las probabilidades en un diccionario
    int_to_char = {i: char for char, i in tokenizer.word_index.items()}
    int_to_char[0] = ''  # Añadir el carácter vacío para el padding
    return {int_to_char[i]: prob for i, prob in enumerate(prediction)}

In [95]:
# Ejemplo de uso
input_text = "I am not familiar with, or knowledgeable about the original languag"
predictions = predict_next_char(model, tokenizer, input_text, seq_length)
print(predictions)

{'': 2.912069e-10, ' ': 0.008098608, 'e': 0.84537244, 't': 0.0050773835, 'o': 0.015000976, 'a': 0.007953188, 'i': 0.062609546, 'n': 0.0002290401, 's': 0.002169434, 'r': 0.00017455738, 'h': 0.004103605, 'l': 0.0011149038, '\n': 0.0007341459, 'd': 1.9611709e-05, 'u': 0.0057491045, 'c': 8.3980376e-05, 'm': 0.0004142076, '.': 0.0015490736, 'y': 0.036456842, 'f': 9.44468e-05, 'g': 3.0248479e-05, 'p': 5.917111e-06, 'w': 6.38095e-05, 'b': 0.000103438884, '>': 2.8584232e-08, 'v': 3.9760475e-06, ',': 0.0011134631, '-': 8.900999e-05, 'I': 4.9225747e-07, 'k': 2.7681324e-05, ':': 4.7664907e-06, 'T': 2.6036204e-09, "'": 6.734473e-05, '"': 9.335563e-05, '*': 5.7025777e-06, 'A': 1.3488265e-07, 'S': 1.1493323e-08, ')': 0.00069851323, '(': 8.5612805e-07, '1': 1.2899555e-07, 'C': 6.7450685e-08, '@': 7.252227e-05, 'R': 1.5494509e-08, 'x': 4.506911e-07, 'M': 2.4733743e-10, 'O': 1.575933e-08, 'N': 4.1148773e-10, 'B': 1.151856e-08, 'j': 8.361986e-07, '^': 3.3197698e-09, 'P': 8.320143e-11, '?': 0.0002837003,

In [90]:
predictions.keys()

dict_keys(['', ' ', 'e', 't', 'o', 'a', 'i', 'n', 's', 'r', 'h', 'l', '\n', 'd', 'u', 'c', 'm', '.', 'y', 'f', 'g', 'p', 'w', 'b', '>', 'v', ',', '-', 'I', 'k', ':', 'T', "'", '"', '*', 'A', 'S', ')', '(', '1', 'C', '@', 'R', 'x', 'M', 'O', 'N', 'B', 'j', '^', 'P', '?', 'L', '0', 'F', 'W', 'H', 'E', '|', 'D', '2', 'G', 'z', '3', '=', '9', 'q', '/', 'U', '5', 'J', '4', '#', 'K', '_', '6', '<', '\\', 'Y', '8', '7', '!', '\t', '[', ']', ';', 'V', 'Q', '+', '~', 'X', '$', 'Z', '`', '&', '%', '}', '{', '\x1b', '\x0c', '\x10'])

In [96]:
max_key = max(predictions, key=predictions.get)
print("Key con mayor valor:", max_key)

Key con mayor valor: e


In [100]:
# Codificar texto de entrada 
def codificar_texto(texto:str):
    text_encoded = []
    for i in range(len(texto)):
        input_text = ''
        if i>0:
            input_text=texto[:i]
        print('Texto a predecir:',input_text)
        predictions = predict_next_char(model, tokenizer, input_text, seq_length)
        max_key = max(predictions, key=predictions.get)
        print('Siguiente caracter más probable:',max_key)
        symbolweights = [(char, weight) for char, weight in predictions.items()]
        codigo_huffman = huffman.codebook(symbolweights)
        print('Caracter a codificar:',texto[i])
        texto_codificado =codigo_huffman[texto[i]]
        print('Texto codificado:',texto_codificado)
        text_encoded.append(texto_codificado)
    text_encoded=''.join(text_encoded)
    return text_encoded
        

In [93]:
def codificar_texto_ascii_binario(texto):
    texto_codificado = ''.join(format(ord(char), '08b') for char in texto)
    return texto_codificado


In [94]:
def codificar_texto_huffman(texto):
    frequencies = Counter(texto)
    symbolweights = [(char, weight) for char, weight in frequencies.items()]
    codigo_huffman = huffman.codebook(symbolweights)
    texto_codificado = ''.join(codigo_huffman[char] for char in texto)
    return texto_codificado

In [73]:
huffman_encoded=codificar_texto_huffman('I am not familiar with, or knowledgeable about the original languag')

'101010 || 111 || 100 || 01011 || 111 || 0010 || 1011 || 0100 || 111 || 00000 || 100 || 01011 || 1100 || 1101 || 1100 || 100 || 0001 || 111 || 10100 || 1100 || 0100 || 00001 || 010101 || 111 || 1011 || 0001 || 111 || 010100 || 0010 || 1011 || 10100 || 1101 || 0011 || 101011 || 0110 || 0011 || 100 || 01111 || 1101 || 0011 || 111 || 100 || 01111 || 1011 || 01110 || 0100 || 111 || 0100 || 00001 || 0011 || 111 || 1011 || 0001 || 1100 || 0110 || 1100 || 0010 || 100 || 1101 || 111 || 1101 || 100 || 0010 || 0110 || 01110 || 100 || 0110'

In [75]:
ascii_encoded=codificar_texto_ascii_binario("I am not familiar with, or knowledgeable about the original languag")

'01001001 || 00100000 || 01100001 || 01101101 || 00100000 || 01101110 || 01101111 || 01110100 || 00100000 || 01100110 || 01100001 || 01101101 || 01101001 || 01101100 || 01101001 || 01100001 || 01110010 || 00100000 || 01110111 || 01101001 || 01110100 || 01101000 || 00101100 || 00100000 || 01101111 || 01110010 || 00100000 || 01101011 || 01101110 || 01101111 || 01110111 || 01101100 || 01100101 || 01100100 || 01100111 || 01100101 || 01100001 || 01100010 || 01101100 || 01100101 || 00100000 || 01100001 || 01100010 || 01101111 || 01110101 || 01110100 || 00100000 || 01110100 || 01101000 || 01100101 || 00100000 || 01101111 || 01110010 || 01101001 || 01100111 || 01101001 || 01101110 || 01100001 || 01101100 || 00100000 || 01101100 || 01100001 || 01101110 || 01100111 || 01110101 || 01100001 || 01100111'

In [101]:
adaptative_encoded=codificar_texto("I am not familiar with, or knowledgeable about the original languag")

Texto a predecir: 
Siguiente caracter más probable:  
Caracter a codificar: I
Texto codificado: 011001
Texto a predecir: I
Siguiente caracter más probable: N
Caracter a codificar:  
Texto codificado: 011
Texto a predecir: I 
Siguiente caracter más probable: (
Caracter a codificar: a
Texto codificado: 0001110111
Texto a predecir: I a
Siguiente caracter más probable:  
Caracter a codificar: m
Texto codificado: 001001010
Texto a predecir: I am
Siguiente caracter más probable: L
Caracter a codificar:  
Texto codificado: 111
Texto a predecir: I am 
Siguiente caracter más probable: g
Caracter a codificar: n
Texto codificado: 1011000
Texto a predecir: I am n
Siguiente caracter más probable: o
Caracter a codificar: o
Texto codificado: 1
Texto a predecir: I am no
Siguiente caracter más probable: n
Caracter a codificar: t
Texto codificado: 11
Texto a predecir: I am not
Siguiente caracter más probable:  
Caracter a codificar:  
Texto codificado: 1
Texto a predecir: I am not 
Siguiente caracter má

'011001011000111011100100101011110110001111101111000000101110000011111100100111000111001100110011010000111111000110111101111001110100111000110111110111111111101111111111011101101010110110110'

In [ ]:
len()

In [102]:
def decodificar_texto_huffman(texto_codificado, codigo_huffman):
    codigo_inverso = {v: k for k, v in codigo_huffman.items()}
    
    texto_decodificado = ''
    temp = ''
    for bit in texto_codificado:
        temp += bit
        if temp in codigo_inverso:
            texto_decodificado += codigo_inverso[temp]
            temp = ''
    
    return texto_decodificado

In [103]:
def decodificador_model(texto_encoded:str):
    texto_decoded = ''
    temp = ''
    codigo_inverso_tmp = {}
    for i in range(len(texto_encoded)):
        if not temp:
            input_text = ''
            if i>0:
                input_text=texto_decoded[:i]
            print('Texto a predecir:',input_text)
            predictions = predict_next_char(model, tokenizer, input_text, seq_length)
            max_key = max(predictions, key=predictions.get)
            print('Siguiente caracter más probable:',max_key)
            symbolweights = [(char, weight) for char, weight in predictions.items()]
            codigo_huffman = huffman.codebook(symbolweights)
            codigo_inverso_tmp = {v: k for k, v in codigo_huffman.items()}
        temp += texto_encoded[i]
        if temp in codigo_inverso_tmp:
            texto_decoded += codigo_inverso_tmp[temp]
            temp = ''
    return texto_decoded

In [104]:
texto_encoded = '011001011000111011100100101011110110001111101111000000101110000011111100100111000111001100110011010000111111000110111101111001110100111000110111110111111111101111111111011101101010110110110'
decodificador_model(texto_encoded)

Texto a predecir: 
Siguiente caracter más probable:  
Texto a predecir: I
Siguiente caracter más probable: N
Texto a predecir: I 
Siguiente caracter más probable: (
Texto a predecir: I a
Siguiente caracter más probable:  
Texto a predecir: I am
Siguiente caracter más probable: L
Texto a predecir: I am 
Siguiente caracter más probable: g
Texto a predecir: I am n
Siguiente caracter más probable: o
Texto a predecir: I am no
Siguiente caracter más probable: n
Texto a predecir: I am not
Siguiente caracter más probable:  
Texto a predecir: I am not 
Siguiente caracter más probable: i
Texto a predecir: I am not f
Siguiente caracter más probable: o
Texto a predecir: I am not fa
Siguiente caracter más probable: c
Texto a predecir: I am not fam
Siguiente caracter más probable: e
Texto a predecir: I am not fami
Siguiente caracter más probable: n
Texto a predecir: I am not famil
Siguiente caracter más probable: l
Texto a predecir: I am not famili
Siguiente caracter más probable: t
Texto a predecir

'I am not familiar with, or knowledgeable about the original languag'